In [1]:
!lsb_release -a
# !cat /etc/shells
# !echo $SHELL
!cat /proc/cpuinfo 
!free -m
!nvidia-smi

%pip install --upgrade pip
%pip install mir_eval librosa h5py
# %pip install torch==1.10.2 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
%pip install note_seq==0.0.3 transformers  scikit-learn  pandas

# !git clone https://github.com/NVIDIA/apex
# %pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" /content/apex/
# # 使用 apex.normalization.FusedRMSNorm 替代 T5LayerNorm 加快计算
# 安装要将近20分钟，还是算了吧

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass

In [2]:
import sys

from google.colab import drive
drive.mount('/content/drive')

%set_env KAGGLE_CONFIG_DIR=/content/drive/MyDrive/kaggle
# 替代export设置kaggle文件夹的位置，路径不能有引号
!kaggle datasets download -d stareven233/maestrov300-hdf5

!rm /content//yui_py37/ -r
!unzip /content/drive/MyDrive/yui_py37.zip -d /content/
sys.path.insert(0, r'/content/yui_py37')
!unzip /content/maestrov300-hdf5.zip -d /content/maestro/
!rm /content/maestrov300-hdf5.zip

# !unzip /content/drive/MyDrive/datasets/maestrov200.zip -d /content/maestro/
# !unzip /content/drive/MyDrive/checkpoints.zip -d /content/drive/MyDrive/

Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/MyDrive/kaggle
100% 18.0G/18.0G [06:32<00:00, 76.9MB/s]
100% 18.0G/18.0G [06:32<00:00, 49.1MB/s]
rm: cannot remove '/content//yui_py37/': No such file or directory
Archive:  /content/drive/MyDrive/yui_py37.zip
  inflating: /content/yui_py37/__init__.py  
   creating: /content/yui_py37/__pycache__/
  inflating: /content/yui_py37/__pycache__/datasets.cpython-39.pyc  
  inflating: /content/yui_py37/__pycache__/event_codec.cpython-39.pyc  
  inflating: /content/yui_py37/__pycache__/note_sequences.cpython-39.pyc  
  inflating: /content/yui_py37/__pycache__/preprocessors.cpython-39.pyc  
  inflating: /content/yui_py37/__pycache__/utils.cpython-39.pyc  
  inflating: /content/yui_py37/__pycache__/vocabularies.cpython-39.pyc  
   creating: /content/yui_py37/config/
  inflating: /content/yui_py37/config/__init__.py  
   creating: /content/yui_py37/config/__pycache__/
  inflating: /content/yui_py37/config/__pycache__/__init__.cpython

In [3]:
# 升级MAESTROv2.0.0 -> 3.0.0 且 输出gpu信息
import utils
# import preprocessors

# preprocessors.upgrade_maestro(r'/content/maestro/')
utils.show_gpu_info()

CUDA GPUs are available
number of avaliable gpu: 1
index of current device: 0
device capability: 3.7
device name: Tesla K80


In [4]:
import os
import time
import logging
import math

import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Config
from transformers.optimization import Adafactor, AdafactorSchedule

from datasets import MaestroDataset3, MaestroSampler2, collate_fn
import vocabularies
import config
from config.data import YuiConfigPro
import utils
from train import train, evaluate

resume = True


# config
cf = YuiConfigPro(
  DATASET_DIR=r'/content/maestro/',
  DATAMETA_NAME=r'maestro-v3.0.0.csv',
  WORKSPACE=r'/content/drive/MyDrive/',
  # WORKSPACE=r'/content/',
  CUDA=True,
  NUM_EPOCHS=20,
  NUM_WORKERS=2,
  BATCH_SIZE=8,
  TRAIN_ITERATION=1500,
)
# 经常分到 k80 显存12GB，batch_size不能太大

# Arugments & parameters
workspace = cf.WORKSPACE
batch_size = cf.BATCH_SIZE
device = torch.device('cuda') if cf.CUDA and torch.cuda.is_available() else torch.device('cpu')
num_workers = cf.NUM_WORKERS

class Adafactor2(Adafactor):
  def __init__(
    self,
    params,
    lr=None,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    scale_parameter=True,
    relative_step=True,
    warmup_init=False,
  ):
    super().__init__(params, lr, eps, clip_threshold, decay_rate, beta1, weight_decay, scale_parameter, relative_step, warmup_init)

  @staticmethod
  def _get_lr(param_group, param_state):
    rel_step_sz = param_group["lr"]
    if param_group["relative_step"]:
      min_step = 1e-6 * param_state["step"] if param_group["warmup_init"] else 1e-3
      exp_lr = math.exp(-(6.45 + param_state["step"] / 3e4))
      # 这个值将在step=[1500,30000]从1.5e-3降到9.6e-4
      rel_step_sz = min(min_step, exp_lr)
    if param_group["scale_parameter"]:
      rel_step_sz *= max(param_group["eps"][1], param_state["RMS"])
    return rel_step_sz


In [5]:
# Checkpoint & Log
# 单独放置，否则多次创建logger会有多个重复输出

checkpoints_dir = os.path.join(workspace, 'checkpoints')
utils.create_folder(checkpoints_dir)
logs_dir = os.path.join(workspace, 'logs')
utils.create_logging(logs_dir, f'train', filemode='w', with_time=True)
resume_checkpoint_path = os.path.join(checkpoints_dir, 'model_resume.pt')
best_checkpoint_path = os.path.join(checkpoints_dir, 'model_best.pt')
statistics_path = os.path.join(checkpoints_dir, 'statistics.pt')

In [6]:
# Codec & Vocabulary
codec = vocabularies.build_codec(cf)
vocabulary = vocabularies.Vocabulary(cf, codec.num_classes, extra_ids=cf.EXTRA_IDS)
t5_config_map = config.build_t5_config(
  d_model=cf.NUM_MEL_BINS,
  vocab_size=vocabulary.vocab_size,
  max_length=cf.MAX_TARGETS_LENGTH,
)
# 简化模型，否则根本训练不动

logging.info(cf)  
if device.type == 'cuda':
  logging.info('Using GPU.')
  logging.info(f'GPU number: {torch.cuda.device_count()}')
else:
  logging.info('Using CPU.')

root: INFO YuiConfigPro(RANDOM_SEED=233, DATASET_DIR='/content/maestro/', DATAMETA_NAME='maestro-v3.0.0.csv', WORKSPACE='/content/drive/MyDrive/', MAX_INPUTS_LENGTH=512, MAX_TARGETS_LENGTH=1024, PROGRAM_GRANULARITY='flat', SAMPLE_RATE=16000, FRAME_SIZE=128, HOP_WIDTH=128, NUM_MEL_BINS=384, FFT_SIZE=2048, MEL_LO_HZ=20.0, MEL_HI_HZ=8000.0, PAD_ID=0, ENCODED_EOS_ID=1, ENCODED_UNK_ID=2, EXTRA_IDS=0, DECODED_EOS_ID=-1, DECODED_INVALID_ID=-2, STEPS_PER_SECOND=100, MAX_SHIFT_SECONDS=10, NUM_VELOCITY_BINS=127, CUDA=True, BATCH_SIZE=8, EXPECT_BATCH_SIZE=128, NUM_WORKERS=2, NUM_EPOCHS=20, TRAIN_ITERATION=1500, LEARNING_RATE=0.001, OVERFIT_PATIENCE=8)
root: INFO Using GPU.
root: INFO GPU number: 1


In [ ]:
torch.cuda.empty_cache()
# 释放显存的cache，可能要多次执行才有效

In [ ]:
# Dataset
meta_path = os.path.join(cf.DATASET_DIR, cf.DATAMETA_NAME)

train_sampler = MaestroSampler2(meta_path, 'train', batch_size=batch_size, config=cf, max_iter_num=cf.TRAIN_ITERATION)
train_dataset = MaestroDataset3(cf.DATASET_DIR, cf, codec, vocabulary, meta_file=cf.DATAMETA_NAME)
train_loader = DataLoader(dataset=train_dataset, batch_sampler=train_sampler, collate_fn=collate_fn, num_workers=num_workers, pin_memory=True)

validate_sampler = MaestroSampler2(meta_path, 'validation', batch_size=batch_size, config=cf, max_iter_num=-1)
validate_loader = DataLoader(dataset=train_dataset, batch_sampler=validate_sampler, collate_fn=collate_fn, num_workers=num_workers, pin_memory=True)
# pin_memory: 锁页内存，不会与虚存进行交换，转到gpu时快一些，但很容易超出gpu显存

# Model
t5_config = T5Config.from_dict(t5_config_map)
logging.info(t5_config)
model = T5ForConditionalGeneration(config=t5_config)
logging.info(f'The model has {model.num_parameters():,} trainable parameters')
# 17,896 for dev; 48,626,048 for pro; while T5-Small has 60 million parameters

# Early stop
early_stopping = utils.EarlyStopping(
  best_path=best_checkpoint_path,
  resume_path=resume_checkpoint_path,
  patience=cf.OVERFIT_PATIENCE, 
  verbose=True
)

# Resume training
resume_epoch = 0
learning_rate = cf.LEARNING_RATE
statistics = {
  'epoch': 0,
  'train_loss': [],
  'eval_loss': []
}

# Loss function
criterion = torch.nn.CrossEntropyLoss(ignore_index=cf.PAD_ID)

# Optimizer
# optimizer = Adafactor(model.parameters(), lr=learning_rate, scale_parameter=False, relative_step=False, warmup_init=False)
optimizer = Adafactor2(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
scheduler = AdafactorSchedule(optimizer, learning_rate)

if not resume:
  ...
  # 从头开始训练模型
elif not os.path.isfile(resume_checkpoint_path):
  logging.info(f'resume_checkpoint_path={resume_checkpoint_path} does not exist, train from scratch')
elif not os.path.isfile(statistics_path):
  logging.info(f'statistics_path={statistics_path} does not exist, train from scratch')
else:
  statistics = torch.load(statistics_path)
  # 单独保存后面数据分析读取方便些
  # raise FileNotFoundError(f'resume_checkpoint_path={resume_checkpoint_path} does not exist')
  checkpoint = torch.load(resume_checkpoint_path)
  # 以TRAIN_ITERATION为单位保存checkpoint
  early_stopping.load_state_dict(checkpoint['early_stopping'])

  model.load_state_dict(checkpoint['model'])
  train_sampler.load_state_dict(checkpoint['sampler'])
  validate_sampler.epoch = train_sampler.epoch
  # 二者epoch一致
  resume_epoch = checkpoint['epoch']
  # scheduler.get_lr 拿到的lr是个列表
  optimizer.load_state_dict(checkpoint['optimizer'])
  logging.info(f'resume training with epoch={resume_epoch}')
  logging.info(f'statistics = {statistics}')

model.to(device)
epoch = resume_epoch
loop_start_time = time.time()
start_time = time.time()
logging.info(f'-------train loop starts, start_time={start_time:.3f}s-------')

# for epoch in range(resume_epoch, cf.NUM_EPOCHS):
while epoch < cf.NUM_EPOCHS:
  train_loss = train(model, device, train_loader, criterion, optimizer, scheduler, accumulation_steps=cf.accumulation_steps)
  statistics['train_loss'].append(train_loss)
  current_lr = scheduler.get_lr()

  # 训练数据完整采样一轮
  if train_sampler.epoch > epoch:
    validate_sampler.reset_state()
    validate_loss = evaluate(model, device, validate_loader, criterion)
    statistics['eval_loss'].append(validate_loss)
    # 等train数据完整过了一遍再进行评估
    logging.info(
      f'epoch={epoch} finish, time={time.time()-start_time:.3f}s, train_loss={train_loss}, validate_loss={validate_loss}'
      f', with lr={current_lr}'
    )

    early_stopping(validate_loss)
    if early_stopping.stop:
      logging.info(f'early stoping')
      break

    epoch += 1
    start_time = time.time()
    train_sampler.reset_state()
  
  # Save model
  statistics['epoch'] = epoch
  checkpoint = {
    'epoch': epoch,
    'model': model.state_dict(),
    'sampler': train_sampler.state_dict(),
    'early_stopping': early_stopping.state_dict(),
    'optimizer': optimizer.state_dict(),
  }
  torch.save(checkpoint, resume_checkpoint_path)
  torch.save(statistics, statistics_path)
  logging.info(f'save model and statistics to {checkpoints_dir}')
logging.info(f'-------train loop ends, time={time.time()-loop_start_time:.3f}s-------')


root: INFO T5Config {
  "bos_token_id": 0,
  "d_ff": 512,
  "d_kv": 64,
  "d_model": 384,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "forced_eos_token_id": 1,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 1024,
  "model_type": "t5",
  "num_beams": 4,
  "num_decoder_layers": 6,
  "num_heads": 6,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "top_p": 0.95,
  "transformers_version": "4.18.0",
  "use_cache": true,
  "vocab_size": 669
}

root: INFO The model has 18,221,184 trainable parameters
root: INFO resume training with epoch=0
root: INFO statistics = {'epoch': 0, 'train_loss': [0.4050339981118837, 0.37911855947971357, 0.36502313115199314, 0.35639949337641413, 0.347767071386178], 'eval_loss': []}
root: INFO -------train loop s

In [ ]:
!rm /content/logs -r